In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_excel('/content/Etiquetado_Juan_David 05092024 (1).xlsx')

In [ ]:
df

,Categoría,Fragmento
0,No se identificó,Ecuador está atravesado un momento difícil. . ...
1,No se identificó,Por eso sean conscientes señor@s y si se enter...
2,No se identificó,"Que grande #Correa. Acá en Colombia, los oposi..."
3,No se identificó,#EcuadorBajoAtaque #ecuador conflicto armado i...
4,No se identificó,@MashiRafael Mashi: ¿Sabe usted por qué me con...
...,...,...
995,No se identificó,Un hommage à ce grand personnage avant de m'en...
996,No se identificó,VAMOS ECUADOR CARAJO! UNIDOS SIN MIEDO! 😠👍🙏🏼💪💪...
997,No se identificó,Heavy militarization on the streets of Ecuador...
998,Desinformación,Este es el gran ejemplo de gobierno que quiere...


In [ ]:
df['Categoría '].value_counts()

,count
Categoría,
No se identificó,870
Desinformación,72
Mal información,45
Mis información,13


## cambiar valores

In [ ]:
df['Categoría '].replace({'Desinformación': 1, 'Mal información': 1,'Mis información':1,'No se identificó ':0}, inplace=True)
#

<ipython-input-5-51e71e0ee86e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Categoría '].replace({'Desinformación': 1, 'Mal información': 1,'Mis información':1,'No se identificó ':0}, inplace=True)
<ipython-input-5-51e71e0ee86e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Categoría 

In [ ]:
df['Categoría '].value_counts()

,count
Categoría,
0,870
1,130


In [ ]:
df.rename(columns={'Categoría ':'label','Fragmento':'text'}, inplace=True)

In [ ]:
# Prepare the data
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        label = self.data.iloc[idx]["label"]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "text": text,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long),
        }


In [ ]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

# Load the pre-trained BERT model and modify the output layer
model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = SentimentDataset(train_data, tokenizer, max_len=256)
val_dataset = SentimentDataset(val_data, tokenizer, max_len=256)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load the pre-trained BERT model and modify the output layer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set hyperparameters and optimizer
epochs = 10
lr = 1e-5
optimizer = AdamW(model.parameters(), lr=lr)


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
# Fine-tune the model
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_steps = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        train_steps += 1

    avg_train_loss = train_loss / train_steps
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_steps = 0
    val_count = 0

    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        val_loss += loss.item()
        val_steps += 1

        _, predicted = torch.max(logits, 1)
        val_count += labels.size(0)
        val_correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / val_steps
    val_accuracy = 100 * val_correct / val_count

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/10, Training Loss: 0.3892
Epoch 1/10, Validation Loss: 0.4043, Validation Accuracy: 83.50%
Epoch 2/10, Training Loss: 0.2985
Epoch 2/10, Validation Loss: 0.4116, Validation Accuracy: 84.50%
Epoch 3/10, Training Loss: 0.1876
Epoch 3/10, Validation Loss: 0.3475, Validation Accuracy: 83.50%
Epoch 4/10, Training Loss: 0.0851
Epoch 4/10, Validation Loss: 0.4693, Validation Accuracy: 84.00%
Epoch 5/10, Training Loss: 0.0437
Epoch 5/10, Validation Loss: 0.5828, Validation Accuracy: 86.50%
Epoch 6/10, Training Loss: 0.0152
Epoch 6/10, Validation Loss: 0.6483, Validation Accuracy: 85.00%
Epoch 7/10, Training Loss: 0.0786
Epoch 7/10, Validation Loss: 0.4830, Validation Accuracy: 86.00%
Epoch 8/10, Training Loss: 0.0382
Epoch 8/10, Validation Loss: 0.4205, Validation Accuracy: 86.50%
Epoch 9/10, Training Loss: 0.0108
Epoch 9/10, Validation Loss: 0.6352, Validation Accuracy: 86.50%
Epoch 10/10, Training Loss: 0.0059
Epoch 10/10, Validation Loss: 0.6662, Validation Accuracy: 85.00%


## Guardar Modelo

In [ ]:
model_path = "fine_tuned_model_desordenes_2"
tokenizer_path = "fine_tuned_tokenizer_desordenes_2"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


## Leer modelo nuevo

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer, BertForSequenceClassification

model_path = "fine_tuned_model_desordenes_2"
tokenizer_path = "fine_tuned_tokenizer_desordenes_2"

tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = encoded_text["input_ids"].to(device)
    attention_mask = encoded_text["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_label = torch.argmax(logits).item()

    return predicted_label


## Usar modelo

In [ ]:

df["predicted_sentiment"] = df["full_text"].apply(lambda x: predict_sentiment(x, model, tokenizer, device))


## Fine tuning Roberta

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

# Prepare the data
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        label = self.data.iloc[idx]["label"]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        return {
            "text": text,
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

# Load the pre-trained RoBERTa model and modify the output layer
model = RobertaForSequenceClassification.from_pretrained("PlanTL-GOB-ES/roberta-base-bne", num_labels=2)

# Split dataset into training and validation
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = SentimentDataset(train_data, tokenizer, max_len=256)
val_dataset = SentimentDataset(val_data, tokenizer, max_len=256)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load the model onto the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set hyperparameters and optimizer
epochs = 3
lr = 1e-5
optimizer = AdamW(model.parameters(), lr=lr)

# Fine-tune the model
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_steps = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_steps += 1

    avg_train_loss = train_loss / train_steps
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_steps = 0
    val_count = 0

    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        val_loss += loss.item()
        val_steps += 1

        _, predicted = torch.max(logits, 1)
        val_count += labels.size(0)
        val_correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / val_steps
    val_accuracy = 100 * val_correct / val_count

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Training Loss: 0.4114
Epoch 1/3, Validation Loss: 0.4572, Validation Accuracy: 83.50%
Epoch 2/3, Training Loss: 0.3042
Epoch 2/3, Validation Loss: 0.3585, Validation Accuracy: 83.50%
Epoch 3/3, Training Loss: 0.1792
Epoch 3/3, Validation Loss: 0.3619, Validation Accuracy: 83.50%


In [ ]:
model_path = "fine_tuned_model_desordenes_2"
tokenizer_path = "fine_tuned_tokenizer_desordenes_2"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


('fine_tuned_tokenizer_desordenes_2/tokenizer_config.json',
 'fine_tuned_tokenizer_desordenes_2/special_tokens_map.json',
 'fine_tuned_tokenizer_desordenes_2/vocab.json',
 'fine_tuned_tokenizer_desordenes_2/merges.txt',
 'fine_tuned_tokenizer_desordenes_2/added_tokens.json')

In [ ]:
import pandas as pd

# Supongamos que df_total es tu DataFrame original
# Paso 1: Agrupar por 'label'
grouped = df.groupby('label')

# Paso 2: Obtener el tamaño de cada grupo
group_sizes = grouped.size()

# Paso 3: Encontrar el tamaño máximo entre los grupos
max_size = group_sizes.max()

# Paso 4: Muestrear cada grupo al tamaño máximo
balanced_df = grouped.apply(lambda x: x.sample(n=max_size, replace=True, random_state=42)).reset_index(drop=True)

# balanced_df ahora es un DataFrame equilibrado

<ipython-input-9-7e4238c2c4e1>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = grouped.apply(lambda x: x.sample(n=max_size, replace=True, random_state=42)).reset_index(drop=True)


In [ ]:
df.label.value_counts()

,count
label,
0,870
1,130


In [ ]:
balanced_df.label.value_counts()

,count
label,
0,870
1,870


In [ ]:
balanced_df

,label,text
0,0,#09Ene 📢Caos en Ecuador: Bandas armadas de nar...
1,0,Mil veces HDLGP Rafael Correa! #EcuadorBajoAtaque
2,0,"Muchas de las cosas q dice Obrador, envejecen ..."
3,0,Hay momentos en la vida que si no se toman acc...
4,0,Con lo de #EcuadorBajoAtaque deberíamos pensar...
...,...,...
1735,1,Tenían tantas ganas de que perdiera el Correis...
1736,1,"Mira Colombia, esto es lo que le pasa a las na..."
1737,1,"Si lo de Ecuador estuviera pasando en Perú, lo..."
1738,1,El Presidente @DanielNoboaOk declaró como grup...


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer, BertForSequenceClassification

model_path = "fine_tuned_model_desordenes_2"
tokenizer_path = "fine_tuned_tokenizer_desordenes_2"

tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = encoded_text["input_ids"].to(device)
    attention_mask = encoded_text["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_label = torch.argmax(logits).item()

    return predicted_label


## Usar modelo

In [ ]:
df

,label,text
0,0,Ecuador está atravesado un momento difícil. . ...
1,0,Por eso sean conscientes señor@s y si se enter...
2,0,"Que grande #Correa. Acá en Colombia, los oposi..."
3,0,#EcuadorBajoAtaque #ecuador conflicto armado i...
4,0,@MashiRafael Mashi: ¿Sabe usted por qué me con...
...,...,...
995,0,Un hommage à ce grand personnage avant de m'en...
996,0,VAMOS ECUADOR CARAJO! UNIDOS SIN MIEDO! 😠👍🙏🏼💪💪...
997,0,Heavy militarization on the streets of Ecuador...
998,1,Este es el gran ejemplo de gobierno que quiere...


In [ ]:

df["predicted_sentiment"] = df["text"].apply(lambda x: predict_sentiment(x, model, tokenizer, device))


In [ ]:
df

,label,text,predicted_sentiment
0,0,Ecuador está atravesado un momento difícil. . ...,0
1,0,Por eso sean conscientes señor@s y si se enter...,0
2,0,"Que grande #Correa. Acá en Colombia, los oposi...",0
3,0,#EcuadorBajoAtaque #ecuador conflicto armado i...,0
4,0,@MashiRafael Mashi: ¿Sabe usted por qué me con...,0
...,...,...,...
995,0,Un hommage à ce grand personnage avant de m'en...,0
996,0,VAMOS ECUADOR CARAJO! UNIDOS SIN MIEDO! 😠👍🙏🏼💪💪...,0
997,0,Heavy militarization on the streets of Ecuador...,0
998,1,Este es el gran ejemplo de gobierno que quiere...,0


In [ ]:
df.groupby('label')['predicted_sentiment'].value_counts()

label  predicted_sentiment
0      0                      858
       1                       12
1      1                       98
       0                       32
Name: count, dtype: int64